# Import libraries 

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import math
import geopandas as gpd
from shapely.geometry import LineString, Point
import numpy as np
import pandas as pd
import geopandas as gpd
import geemap
import ee
import time
from datetime import datetime, timedelta
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import seaborn as sns
from rasterio.plot import show
from rasterio.merge import merge
import rasterio as rio
from pathlib import Path
from lifelines import KaplanMeierFitter

from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.feature_selection import RFE

# Sign up for google earth engine
# https://accounts.google.com/v3/signin/identifier?continue=https%3A%2F%2Fuc.appengine.google.com%2F_ah%2Fconflogin%3Fstate%3D%7EAJKiYcGTAqfnLHQ3IF0PRvxyDVDHOzQi3yIk92Z-tP7i6cyDz1qVoE4NaKkTL5gVN9MtRNpdCih_DPZ3-cleXMSfg7lbQn5M6ZFIawvFwRVMuRY-XUXstvtr8yOEmBXZxVFwrDS2MfySBsbdR-TyXZjp5m3DySECt4X2Zn55Y8Kj3Fg4INJl9ztrjc6twcmkkvtM35aqc23L&ifkv=ARZ0qKJ119_csxBqwfHRZLfiV0_DiUA_T-yvNLfs8hD5uHR4WTitbIsQ_us3PSxBY1yOnVB9TLuqug&passive=true&flowName=GlifWebSignIn&flowEntry=ServiceLogin&dsh=S-1895545422%3A1712087817582842&theme=mn&ddm=0

ee.Authenticate()

ee.Initialize()

# Visualize data

In [ ]:
### Graph of fire perimeter - 4days

import geopandas as gpd
import matplotlib.pyplot as plt

# Read in the shapefile
file = "C:\\PATH\\TO\\FIRESPREAD\\SHP\\FireSpreadPoly_4days_simple.shp"
data = gpd.read_file(file)

# Sort by date
data_sorted = data.sort_values('GDB_TO_DAT')
xmin, ymin, xmax, ymax = data_sorted.total_bounds
# Create a 2x2 subplot figure
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

# Loop over each day (0-3) and plot the fire spread on the corresponding subplot
for i, ax in enumerate(axs.flatten()):
    # Subset the data to only include the current day
    data_subset = data_sorted.iloc[[i]]
    # Plot the fire spread on the current subplot
    data_subset.plot(column='GDB_TO_DAT', alpha=0.9,color="red", ax=ax)
    # Set the title of the subplot to the current day
    ax.set_title(f"Day {i}")
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    ax.set_xlabel("Longitude")
    # Label the y-axis as "Latitude"
    ax.set_ylabel("Latitude")

# Adjust the spacing between subplots
plt.tight_layout()

#plt.savefig("C:\\PATH\\TO\\SAVE\\IMAGE\\fourpanel_fireprogression.jpg")

# Show the plot
plt.show()


# Visualize the identification of held and breached

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import math

# Day current 
# Day Next
i=1
w=2
# buffer to remove edge effects 
meters = 300

#########################
file = "C:\\PATH\\TO\\FIRESPREAD\\SHP\\FireSpreadPoly_4days_simple.shp"
# Read in the shapefile 
data = gpd.read_file(file)

# Subset the data GeoDataFrame to only include the first row
data_subset = data.iloc[[w]]

xmin, ymin, xmax, ymax = data.total_bounds
latitude = (ymin + ymax) / 2
decimal_degrees = meters / (111319.9 * math.cos(latitude))

# Negative buffer the data subset by 100 meters
data_buffer = data_subset.buffer(decimal_degrees)

# Convert polygon to polyline
dataline = gpd.read_file(file)
polyline = dataline.boundary

# Subset the polyline GeoDataFrame to only include the first row
polyline_subset = polyline.iloc[[i]]

# Clip the polyline to the negative buffer of the data subset
polyline_clip_breach = polyline_subset.intersection(data_buffer.geometry.iloc[0])

# Remove the overlapping part between polyline_subset and polyline_clip_breach
polyline_clip_held = polyline_subset.difference(polyline_clip_breach)

# Create a 3x2 subplot figure
fig, axs = plt.subplots(2, 2, figsize=(18, 12))

# Plot the polyline on the top subplot
axs[0, 0].set_aspect(1)
polyline_subset.plot(ax=axs[0, 0], color='black')
axs[0, 0].set_title("Polyline - Current fire perimeter")
axs[0, 0].set_xlim([xmin, xmax])
axs[0, 0].set_ylim([ymin, ymax])

# Plot the polygon on the middle subplot
axs[0, 1].set_aspect(1)
data_subset.plot(ax=axs[0, 1], color='black')
axs[0, 1].set_title("Polygon - Next day fire perimeter")
axs[0, 1].set_xlim([xmin, xmax])
axs[0, 1].set_ylim([ymin, ymax])

# Plot the clipped polyline on the bottom-left subplot
axs[1, 0].set_aspect(1)
polyline_clip_breach.plot(ax=axs[1, 0], color='red')
axs[1, 0].set_title("Polyline - Breached")
axs[1, 0].set_xlim([xmin, xmax])
axs[1, 0].set_ylim([ymin, ymax])

# Plot the held polyline on the bottom-right subplot
axs[1, 1].set_aspect(1)
polyline_clip_held.plot(ax=axs[1, 1], color='green')
axs[1, 1].set_title("Polyline - Held")
axs[1, 1].set_xlim([xmin, xmax])
axs[1, 1].set_ylim([ymin, ymax])

for ax in axs.flatten():
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    
#plt.savefig("C:\\PATH\\TO\\SAVE\\IMAGE\\fourpanel_breachheld.jpg")

# Show the plot
plt.show()


# Identify breached points 

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

### CHANGE THIS ###
PathToOutputShp = "C:\\Users\\PATH\\TO\\OUTPUT\\SHP\\polyline_breach_PTS.shp"

# the approximate distance between each point (estimated based on decimal degree conversion)
z=620

# Convert MultiLineString to single LineString
polyline_gdf = polyline_clip_breach.explode().reset_index(drop=True)

# Create an empty list to store the points GeoDataFrames
points_gdfs = []

# Loop through the geometries in the GeoDataFrame
for polyline_geometry in polyline_gdf.geometry:

    # Create a LineString object from the polyline geometry
    polyline_ls = LineString(polyline_geometry)

    # Calculate the length of the line string in meters
    line_length = polyline_ls.length
    
    # Trim short line segments blips
    if line_length <= 0.01:
        continue

    # Calculate the bounds of the line string
    x_coords = [p[0] for p in polyline_ls.coords]
    y_coords = [p[1] for p in polyline_ls.coords]
    xmin, ymin, xmax, ymax = min(x_coords), min(y_coords), max(x_coords), max(y_coords)
    
    latitude = (ymin + ymax) / 2
    meters = z
    #decimal_degrees = -(meters / (111319.9 * math.cos(latitude)))
    decimal_degrees = meters / (111319.9 * np.sqrt(1 - 0.00669438 * np.sin(latitude)**2))
    # Calculate the number of points needed to space them every 60 meters
    #num_points = int(line_length // 0.001) + 1

    # Create a list of points spaced every 60 meters along the line string
    #points = [polyline_ls.interpolate(i * 0.001) for i in range(num_points)]
    
    # Calculate the number of points needed to space them every 60 meters
    num_points = int(line_length // decimal_degrees) + 1

    # Create a list of points spaced every 60 meters along the line string
    points = [polyline_ls.interpolate(i * decimal_degrees) for i in range(num_points)]

    # Convert the points to a GeoDataFrame
    points_gdf = gpd.GeoDataFrame(geometry=[Point(p.x, p.y) for p in points], crs=polyline_gdf.crs)

    # Append the points GeoDataFrame to the list
    points_gdfs.append(points_gdf)

# Concatenate all the points GeoDataFrames into a single GeoDataFrame
points_gdf = pd.concat(points_gdfs)

# Save the points as a shapefile
points_gdf.to_file(PathToOutputShp)
print("pts complete")

# Set up the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the points
points_gdf.plot(ax=ax, marker='o', color='red', markersize=5)

# Optionally, plot the original line for context
polyline_gdf.plot(ax=ax, linewidth=1, color='blue')

# Set the title and axes labels
ax.set_title('Points spaced along polyline')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Show the plot
plt.show()


# Identify held points 

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import numpy as np
import pandas as pd
import geopandas as gpd

### CHANGE THIS ###
PathToOutputShp = "C:\\Users\\PATH\\TO\\OUTPUT\\SHP\\polyline_held_PTS.shp"

# the approximate distance between each point
z=620

# Convert MultiLineString to single LineString
polyline_gdf = polyline_clip_held.explode().reset_index(drop=True)

# Create an empty list to store the points GeoDataFrames
points_gdfs = []

# Loop through the geometries in the GeoDataFrame
for polyline_geometry in polyline_gdf.geometry:

    # Create a LineString object from the polyline geometry
    polyline_ls = LineString(polyline_geometry)

    # Calculate the length of the line string in meters
    line_length = polyline_ls.length
    
    # Trim short line segments blips
    if line_length <= 0.01:
        continue

    # Calculate the bounds of the line string
    x_coords = [p[0] for p in polyline_ls.coords]
    y_coords = [p[1] for p in polyline_ls.coords]
    xmin, ymin, xmax, ymax = min(x_coords), min(y_coords), max(x_coords), max(y_coords)
    
    latitude = (ymin + ymax) / 2
    meters = z
    #decimal_degrees = -(meters / (111319.9 * math.cos(latitude)))
    decimal_degrees = meters / (111319.9 * np.sqrt(1 - 0.00669438 * np.sin(latitude)**2))
    # Calculate the number of points needed to space them every 60 meters
    #num_points = int(line_length // 0.001) + 1

    # Create a list of points spaced every 60 meters along the line string
    #points = [polyline_ls.interpolate(i * 0.001) for i in range(num_points)]
    
    # Calculate the number of points needed to space them every 60 meters
    num_points = int(line_length // decimal_degrees) + 1

    # Create a list of points spaced every 60 meters along the line string
    points = [polyline_ls.interpolate(i * decimal_degrees) for i in range(num_points)]

    # Convert the points to a GeoDataFrame
    points_gdf = gpd.GeoDataFrame(geometry=[Point(p.x, p.y) for p in points], crs=polyline_gdf.crs)

    # Append the points GeoDataFrame to the list
    points_gdfs.append(points_gdf)

# Concatenate all the points GeoDataFrames into a single GeoDataFrame
points_gdf = pd.concat(points_gdfs)

# Save the points as a shapefile
points_gdf.to_file(PathToOutputShp)
print("pts complete")

# Set up the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the points
points_gdf.plot(ax=ax, marker='o', color='red', markersize=5)

# Optionally, plot the original line for context
polyline_gdf.plot(ax=ax, linewidth=1, color='blue')

# Set the title and axes labels
ax.set_title('Points spaced along polyline')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Show the plot
plt.show()



# Weather and topograph data collection 

In [ ]:
file = "C:\\PATH\\TO\\FIRESPREAD\\SHP\\FireSpreadPoly_4days_simple.shp"

datalength = len(gpd.read_file(file))
print(int(datalength))

i=0
w=1

df_list_breach = []
df_list_held = []

while w < int(datalength):
    # Day current 
    # Day Next

    # temp path for ee library, this was created above
    PathToOutputShpBreach = "C:\\Users\\PATH\\TO\\OUTPUT\\SHP\\polyline_breach_PTS.shp"
    PathToOutputShpHeld = "C:\\Users\\PATH\\TO\\OUTPUT\\SHP\\polyline_held_PTS.shp"

    # buffer to remove edge effects 
    meters = 300

    # the approximate distance (m) between each point along the perimeter
    z=620

    #############################################################################################################################
    #############################################################################################################################
    #############################################################################################################################
    # identify breach and held locations from i to w
    # start time
    start_time = time.time()

    #########################
    # Read in the shapefile #
    data = gpd.read_file(file)
    data = data.sort_values('GDB_TO_DAT')

    # Subset the data GeoDataFrame to only include the first row
    data_subset = data.iloc[[w]]

    xmin, ymin, xmax, ymax = data.total_bounds
    latitude = (ymin + ymax) / 2
    decimal_degrees = meters / (111319.9 * math.cos(latitude))

    # Negative buffer the data subset by 100 meters
    data_buffer = data_subset.buffer(decimal_degrees)

    # Convert polygon to polyline
    dataline = gpd.read_file(file)
    polyline = dataline.boundary

    # Subset the polyline GeoDataFrame to only include the first row
    polyline_subset = polyline.iloc[[i]]

    # Clip the polyline to the negative buffer of the data subset
    polyline_clip_breach = polyline_subset.intersection(data_buffer.geometry.iloc[0])

    # Remove the overlapping part between polyline_subset and polyline_clip_breach
    polyline_clip_held = polyline_subset.difference(polyline_clip_breach)

    #############################################################################################################################
    #############################################################################################################################
    # polyline_breach_PTS processing

    # Convert MultiLineString to single LineString
    polyline_gdf = polyline_clip_breach.explode().reset_index(drop=True)

    # Create an empty list to store the points GeoDataFrames
    points_gdfs = []

    # Loop through the geometries in the GeoDataFrame
    for polyline_geometry in polyline_gdf.geometry:

        # Create a LineString object from the polyline geometry
        polyline_ls = LineString(polyline_geometry)

        # Calculate the length of the line string in meters
        line_length = polyline_ls.length

        # Trim short line segments blips
        if line_length <= 0.01:
            continue

        # Calculate the bounds of the line string
        x_coords = [p[0] for p in polyline_ls.coords]
        y_coords = [p[1] for p in polyline_ls.coords]
        xmin, ymin, xmax, ymax = min(x_coords), min(y_coords), max(x_coords), max(y_coords)

        latitude = (ymin + ymax) / 2
        meters = z
        #decimal_degrees = -(meters / (111319.9 * math.cos(latitude)))
        decimal_degrees = meters / (111319.9 * np.sqrt(1 - 0.00669438 * np.sin(latitude)**2))
        # Calculate the number of points needed to space them every 60 meters
        #num_points = int(line_length // 0.001) + 1

        # Create a list of points spaced every 60 meters along the line string
        #points = [polyline_ls.interpolate(i * 0.001) for i in range(num_points)]

        # Calculate the number of points needed to space them every 60 meters
        num_points = int(line_length // decimal_degrees) + 1

        # Create a list of points spaced every 60 meters along the line string
        points = [polyline_ls.interpolate(i * decimal_degrees) for i in range(num_points)]

        # Convert the points to a GeoDataFrame
        points_gdf = gpd.GeoDataFrame(geometry=[Point(p.x, p.y) for p in points], crs=polyline_gdf.crs)

        # Append the points GeoDataFrame to the list
        points_gdfs.append(points_gdf)

    # Concatenate all the points GeoDataFrames into a single GeoDataFrame
    points_gdf = pd.concat(points_gdfs)

    # Save the points as a shapefile
    points_gdf.to_file(PathToOutputShpBreach)
    print("pts complete")
    #############################################################################################################################
    #############################################################################################################################
    #
    ##
    ###
    ##
    #
    #############################################################################################################################
    #############################################################################################################################
    # polyline_held_PTS processing

    # Convert MultiLineString to single LineString
    polyline_gdf = polyline_clip_held.explode().reset_index(drop=True)

    # Create an empty list to store the points GeoDataFrames
    points_gdfs = []

    # Loop through the geometries in the GeoDataFrame
    for polyline_geometry in polyline_gdf.geometry:

        # Create a LineString object from the polyline geometry
        polyline_ls = LineString(polyline_geometry)

        # Calculate the length of the line string in meters
        line_length = polyline_ls.length

        # Trim short line segments blips
        if line_length <= 0.01:
            continue

        # Calculate the bounds of the line string
        x_coords = [p[0] for p in polyline_ls.coords]
        y_coords = [p[1] for p in polyline_ls.coords]
        xmin, ymin, xmax, ymax = min(x_coords), min(y_coords), max(x_coords), max(y_coords)

        latitude = (ymin + ymax) / 2
        meters = z
        #decimal_degrees = -(meters / (111319.9 * math.cos(latitude)))
        decimal_degrees = meters / (111319.9 * np.sqrt(1 - 0.00669438 * np.sin(latitude)**2))
        # Calculate the number of points needed to space them every 60 meters
        #num_points = int(line_length // 0.001) + 1

        # Create a list of points spaced every 60 meters along the line string
        #points = [polyline_ls.interpolate(i * 0.001) for i in range(num_points)]

        # Calculate the number of points needed to space them every 60 meters
        num_points = int(line_length // decimal_degrees) + 1

        # Create a list of points spaced every 60 meters along the line string
        points = [polyline_ls.interpolate(i * decimal_degrees) for i in range(num_points)]

        # Convert the points to a GeoDataFrame
        points_gdf = gpd.GeoDataFrame(geometry=[Point(p.x, p.y) for p in points], crs=polyline_gdf.crs)

        # Append the points GeoDataFrame to the list
        points_gdfs.append(points_gdf)

    # Concatenate all the points GeoDataFrames into a single GeoDataFrame
    points_gdf = pd.concat(points_gdfs)

    # Save the points as a shapefile
    points_gdf.to_file(PathToOutputShpHeld)
    print("pts complete")
    
    #############################################################################################################################
    #############################################################################################################################
    
    
    
    #############################################################################################################################
    #############################################################################################################################
    ee_fc_breach = geemap.shp_to_ee(PathToOutputShpBreach)
    ee_fc_held = geemap.shp_to_ee(PathToOutputShpHeld)
    #############################################################################################################################
    #############################################################################################################################
    # terrain breach
    # Load the 3DEP dataset  and others and filter by the extent of the points
    dem = ee.Image("USGS/3DEP/10m").clip(ee_fc_breach.geometry().bounds())
    topo_diversity = ee.Image("CSP/ERGo/1_0/US/topoDiversity").clip(ee_fc_breach.geometry().bounds())
    physiography = ee.Image("CSP/ERGo/1_0/US/physiography").clip(ee_fc_breach.geometry().bounds())
    physio_diversity = ee.Image("CSP/ERGo/1_0/US/physioDiversity").clip(ee_fc_breach.geometry().bounds())
    mtpi = ee.Image("CSP/ERGo/1_0/US/mTPI").clip(ee_fc_breach.geometry().bounds())
    lithology = ee.Image("CSP/ERGo/1_0/US/lithology").clip(ee_fc_breach.geometry().bounds())
    landforms = ee.Image("CSP/ERGo/1_0/US/landforms").clip(ee_fc_breach.geometry().bounds())
    chili = ee.Image("CSP/ERGo/1_0/US/CHILI").clip(ee_fc_breach.geometry().bounds())

    # Extract data for each image at appropriate scale
    terrain_fc = ee.Terrain.products(dem).sampleRegions(collection=ee_fc_breach, scale=10)

    topo_diversity_fc = topo_diversity.sampleRegions(collection=ee_fc_breach, scale=90)
    physiography_fc = physiography.sampleRegions(collection=ee_fc_breach, scale=90)
    physio_diversity_fc = physio_diversity.sampleRegions(collection=ee_fc_breach, scale=270)
    mtpi_fc = mtpi.sampleRegions(collection=ee_fc_breach, scale=270)
    lithology_fc = lithology.sampleRegions(collection=ee_fc_breach, scale=90)
    landforms_fc = landforms.sampleRegions(collection=ee_fc_breach, scale=10)
    chili_fc = chili.sampleRegions(collection=ee_fc_breach, scale=10)

    # Convert the elevation and terrain product data to Pandas DataFrames
    terrain_df = geemap.ee_to_df(terrain_fc)

    topo_diversity_df = geemap.ee_to_df(topo_diversity_fc)
    physiography_df = geemap.ee_to_df(physiography_fc)
    physio_diversity_df = geemap.ee_to_df(physio_diversity_fc)
    mtpi_df = geemap.ee_to_df(mtpi_fc)
    lithology_df = geemap.ee_to_df(lithology_fc)
    landforms_df = geemap.ee_to_df(landforms_fc)
    chili_df = geemap.ee_to_df(chili_fc)

    # Rename columns to appropriate names
    topo_diversity_df = topo_diversity_df.rename(columns={'constant': 'topo_diversity'})
    physiography_df = physiography_df.rename(columns={'constant': 'physiography'})
    physio_diversity_df = physio_diversity_df.rename(columns={'b1': 'physio_diversity'})
    mtpi_df = mtpi_df.rename(columns={'elevation': 'mtpi'})
    lithology_df = lithology_df.rename(columns={'b1': 'lithology'})
    landforms_df = landforms_df.rename(columns={'constant': 'landforms'})
    chili_df = chili_df.rename(columns={'constant': 'chili'})
    
    ##########################
    ##### Breach Weather #####
    ##########################

    start_date_str = str(data.GDB_TO_DAT[i])
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = start_date + timedelta(days=1)

    era5 = ee.ImageCollection("ECMWF/ERA5/HOURLY").select(['u_component_of_wind_10m', 'v_component_of_wind_10m', 
                                                      'dewpoint_temperature_2m', 'temperature_2m', 'skin_temperature',
                                                      'soil_temperature_level_1', 'volumetric_soil_water_layer_1',
                                                      'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3',
                                                      'volumetric_soil_water_layer_4', 'forecast_albedo',
                                                      'surface_latent_heat_flux', 'surface_net_solar_radiation',
                                                      'surface_net_thermal_radiation',
                                                      'potential_evaporation',
                                                      'total_precipitation', 'leaf_area_index_high_vegetation',
                                                      'leaf_area_index_low_vegetation']).filterDate(start_date, end_date)

    
    era5mean_fc = era5.mean().reduceRegions(collection=ee_fc_breach, reducer=ee.Reducer.mean(), scale=11132)
    era5mean_df = geemap.ee_to_df(era5mean_fc)

    print(era5mean_df)
    
    
    ##########################
    ### END Breach Weather ###
    ##########################

    df = pd.concat([terrain_df, topo_diversity_df.iloc[:, 1], physiography_df.iloc[:, 1], physio_diversity_df.iloc[:, 1], 
                mtpi_df.iloc[:, 1], lithology_df.iloc[:, 1], landforms_df.iloc[:, 1], chili_df.iloc[:, 1], era5mean_df], axis=1)
    df_breach = df.dropna()
    df_breach["date"] = data.GDB_TO_DAT[i]
    df_breach["numberofpoints"] = len(df_breach)

    print("df_breach length is: ", len(df_breach))
    #df_breach
    
    #############################################################################################################################
    #############################################################################################################################
    # terrain held
    # Load the 3DEP dataset and filter by the extent of the points
    dem = ee.Image("USGS/3DEP/10m").clip(ee_fc_held.geometry().bounds())
    topo_diversity = ee.Image("CSP/ERGo/1_0/US/topoDiversity").clip(ee_fc_held.geometry().bounds())
    physiography = ee.Image("CSP/ERGo/1_0/US/physiography").clip(ee_fc_held.geometry().bounds())
    physio_diversity = ee.Image("CSP/ERGo/1_0/US/physioDiversity").clip(ee_fc_held.geometry().bounds())
    mtpi = ee.Image("CSP/ERGo/1_0/US/mTPI").clip(ee_fc_held.geometry().bounds())
    lithology = ee.Image("CSP/ERGo/1_0/US/lithology").clip(ee_fc_held.geometry().bounds())
    landforms = ee.Image("CSP/ERGo/1_0/US/landforms").clip(ee_fc_held.geometry().bounds())
    chili = ee.Image("CSP/ERGo/1_0/US/CHILI").clip(ee_fc_held.geometry().bounds())

    # Extract data for each image at appropriate scale
    terrain_fc = ee.Terrain.products(dem).sampleRegions(collection=ee_fc_held, scale=10)

    topo_diversity_fc = topo_diversity.sampleRegions(collection=ee_fc_held, scale=90)
    physiography_fc = physiography.sampleRegions(collection=ee_fc_held, scale=90)
    physio_diversity_fc = physio_diversity.sampleRegions(collection=ee_fc_held, scale=270)
    mtpi_fc = mtpi.sampleRegions(collection=ee_fc_held, scale=270)
    lithology_fc = lithology.sampleRegions(collection=ee_fc_held, scale=90)
    landforms_fc = landforms.sampleRegions(collection=ee_fc_held, scale=10)
    chili_fc = chili.sampleRegions(collection=ee_fc_held, scale=10)

    # Convert the elevation and terrain product data to Pandas DataFrames
    terrain_df = geemap.ee_to_df(terrain_fc)

    topo_diversity_df = geemap.ee_to_df(topo_diversity_fc)
    physiography_df = geemap.ee_to_df(physiography_fc)
    physio_diversity_df = geemap.ee_to_df(physio_diversity_fc)
    mtpi_df = geemap.ee_to_df(mtpi_fc)
    lithology_df = geemap.ee_to_df(lithology_fc)
    landforms_df = geemap.ee_to_df(landforms_fc)
    chili_df = geemap.ee_to_df(chili_fc)

    # Rename columns to appropriate names
    topo_diversity_df = topo_diversity_df.rename(columns={'constant': 'topo_diversity'})
    physiography_df = physiography_df.rename(columns={'constant': 'physiography'})
    physio_diversity_df = physio_diversity_df.rename(columns={'b1': 'physio_diversity'})
    mtpi_df = mtpi_df.rename(columns={'elevation': 'mtpi'})
    lithology_df = lithology_df.rename(columns={'b1': 'lithology'})
    landforms_df = landforms_df.rename(columns={'constant': 'landforms'})
    chili_df = chili_df.rename(columns={'constant': 'chili'})

    ##########################
    ##### Held Weather #####
    ##########################

    start_date_str = str(data.GDB_TO_DAT[i])
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = start_date + timedelta(days=1)

    era5 = ee.ImageCollection("ECMWF/ERA5/HOURLY").select(['u_component_of_wind_10m', 'v_component_of_wind_10m', 
                                                      'dewpoint_temperature_2m', 'temperature_2m', 'skin_temperature',
                                                      'soil_temperature_level_1', 'volumetric_soil_water_layer_1',
                                                      'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3',
                                                      'volumetric_soil_water_layer_4', 'forecast_albedo',
                                                      'surface_latent_heat_flux', 'surface_net_solar_radiation',
                                                      'surface_net_thermal_radiation',
                                                      'potential_evaporation',
                                                      'total_precipitation', 'leaf_area_index_high_vegetation',
                                                      'leaf_area_index_low_vegetation']).filterDate(start_date, end_date)

    
    era5mean_fc = era5.mean().reduceRegions(collection=ee_fc_held, reducer=ee.Reducer.mean(), scale=11132)
    era5mean_df = geemap.ee_to_df(era5mean_fc)

    ##########################
    ### END Held Weather ###
    ##########################
    
    #dfs = [terrain_df, topo_diversity_df, physiography_df, physio_diversity_df, mtpi_df, lithology_df, landforms_df, chili_df]
    df = pd.concat([terrain_df, topo_diversity_df.iloc[:, 1], physiography_df.iloc[:, 1], physio_diversity_df.iloc[:, 1], 
                    mtpi_df.iloc[:, 1], lithology_df.iloc[:, 1], landforms_df.iloc[:, 1], chili_df.iloc[:, 1], era5mean_df], axis=1)

    df_held = df.dropna()
    df_held["date"] = data.GDB_TO_DAT[i]
    df_held["numberofpoints"] = len(df_held)


    print("df_held length is: ",len(df_held))
    end_time = time.time()
    total_time = end_time - start_time
    print("Total time:", total_time, " seconds")
    
    i=i+1
    w=w+1
    
    df_list_held.append(df_held)
    df_list_breach.append(df_breach)


held_df = pd.concat(df_list_held, ignore_index=True)
breach_df = pd.concat(df_list_breach, ignore_index=True)

held_df['binary'] = 0
breach_df['binary'] = 1
df = pd.concat([held_df, breach_df], axis=0)

print("finished")

# Survival Analysis Using Kaplan-Meier Estimator

In [ ]:
from lifelines import KaplanMeierFitter
import pandas as pd

reference_date = pd.to_datetime(df['date']).min()  # This sets the earliest date in your data as a reference point.
print(reference_date)
df['duration'] = (pd.to_datetime(df['date']) - reference_date).dt.days  # This calculates the duration in days from the reference date.


# Create a Kaplan-Meier estimator
kmf = KaplanMeierFitter()

# Fit the Kaplan-Meier estimator using the 'binary' column as the event indicator
kmf.fit(df['duration'], event_observed=df['binary'], label='Kaplan-Meier Estimate')

# Plot the Kaplan-Meier survival curve
kmf.plot()


# Cox Proportional Hazards Model Analysis

In [ ]:
import pandas as pd
from lifelines import CoxPHFitter

# Assuming 'df' is already defined and contains your data
# Convert 'date' from a datetime to a numeric type representing the duration
reference_date = pd.to_datetime(df['date']).min()
df['duration'] = (pd.to_datetime(df['date']) - reference_date).dt.days

# 'binary' column is already set up correctly as the event indicator

# Select the features to include in the Cox model
# Add or remove columns as necessary for your analysis
columns = [
    'aspect', 'elevation', 'u_component_of_wind_10m','surface_latent_heat_flux',
    'v_component_of_wind_10m', 'duration', 'chili', 'dewpoint_temperature_2m',
     'leaf_area_index_high_vegetation', 'leaf_area_index_low_vegetation','binary'
]

# Ensure all columns are included in the DataFrame before fitting the model
X = df[columns]

# Initialize and fit the Cox Proportional Hazards model
cph = CoxPHFitter()
cph.fit(X, duration_col='duration', event_col='binary', show_progress=True)

# Print the summary of the Cox model
cph.summary

# Plot the survival curves for the Cox model
cph.plot()
plt.show()